This notebook is optmizied to be executed on Google Colab (https://colab.research.google.com).

Here, the U-net deep learning model creates a segmentation map (mask) on new images. The results are compared to an expert segmentation.

Example: image (l) and mask (r) \\
    <img src="https://raw.githubusercontent.com/matjesg/DeepFLaSH/master/assets/cFOS_Wue.png" width="250" height="250" alt="cFOS_Wue">
    <img src="https://raw.githubusercontent.com/matjesg/DeepFLaSH/master/assets/cFOS_Wue_mask.png" width="250" height="250" alt="cFOS_Wue_mask">

Press the the *play* butten to execute the cells. It will show up between \[     \] on the left side of the code cells.

In [ ]:
# Set up Google Colab Environment
!git clone https://github.com/matjesg/DeepFLaSH.git
import os
import sys
ROOT_DIR = os.path.abspath("DeepFLaSH")
sys.path.append(ROOT_DIR)\
    
import numpy as np
from unet import utils
from unet import colab_utils
from google.colab import files

# Load model
model = utils.load_unet('cFOS')

# Load images
img_names, img_list = colab_utils.load_samples(path = 'cFOS', suffix = 'cFOS')
msk_names, msk_list = colab_utils.load_samples(path = 'cFOS', suffix = 'expert')

# Predict masks with the unet
pred_train = model.predict(np.asarray(img_list))
msk_train_list = [pred_train[i] for i in range(pred_train.shape[0])]

# Calculate Jaccard Similarity
jac_sim = utils.jaccard_sim(msk_list, pred_train, threshold = 0.94)

**Plot results**
The joined mask consists of the manual/expert segmentation mask and unet prediction.

Color code: 
- white = merge
- magenta = unet only
- green = original/expert only

In [ ]:
join_list = [utils.join_masks(msk_train_list[i], msk_list[i], threshold = 0.94) for i in range(len(msk_list))]
utils.plot_image_and_mask(img_names = img_names, img_list = img_list,
                          msk_names = jac_sim, msk_list = join_list,
                          msk_head = 'Jaccard Similarity')